### RAG (Retrieval-Augmented Generation)
> 학습된 지식 + 실시간으로 외부 데이터를 검색하고 활용해 더 정확한 답변 생성

 기존의 대규모 언어 모델(LLM)을 확장하여, 주어진 질문에 대해 좀 더 정확하고 풍부한 답변을 제공하기 위한 방법입니다. 이 기법은 검색(Retrieval)과 생성(Generation) 과정을 결합하여, 모델이 학습된 지식만을 바탕으로 답변을 생성하는 것이 아니라, 실시간으로 외부 데이터를 검색하여 그 정보를 활용하여 더 정확한 답변을 생성

 - **검색 단계 (Retrieval)**
    
    사용자의 질문이나 컨텍스트에 맞는 외부 데이터를 실시간으로 검색합니다. 이 데이터는 모델이 답변을 생성하는 데 필요한 **자세하고 정확한 정보**를 포함하고 있어야 하며, 여러 문서 중 적합한 정보를 선택하는 과정이 포함됩니다.
    
    **주요 구성 요소:**
    
    - **문서 인덱싱**: 외부 데이터베이스나 문서를 검색 가능한 형태로 변환
    - **임베딩 생성**: 문서와 질문을 벡터 공간에서 표현
    - **유사도 계산**: 질문과 관련성이 높은 문서를 찾기 위한 유사도 측정
    - **상위 K개 문서 선택**: 가장 관련성이 높은 문서들을 선별
- **생성 단계 (Generation)**
    
    검색된 정보를 기반으로 LLM이 사용자의 질문에 대해 자연스럽고 정확한 답변을 생성합니다. 이 과정에서 모델은 **검색된 데이터와 기존의 지식을 결합**하여 답변을 만들며, 생성된 답변은 사용자가 필요로 하는 정보와 맥락을 고려하여 정확히 맞추어집니다.

- 장점: 실시간 정보 반영, 정확한 답변, 할루시네이션 방지
- 단점: 검색 품질 의존성, 지연 시간, 복잡성 증가, 컨텍스트 길이 제한

### RAG의 주요 구현 기술

- 벡터 데이터베이스
    - **Pinecone** : 관리형 벡터 데이터베이스 서비스 ( https://www.pinecone.io/ )
    - **Weaviate** : 오픈소스 벡터 데이터베이스 ( https://weaviate.io/ )
    - **Chroma** : 경량화 된 벡터 데이터베이스 ( https://www.trychroma.com/ )
    - **FAISS** : Facebook이 개발한 유사도 검색 라이브러리 ( https://faiss.ai/ )
- 임베딩 모델
    - **Sentence-BERT** : 문장 임베딩 특화 모델 ( https://sbert.net/ )
    - **OpenAI Embeddings** : GPT 기반 임베딩 모델
        - https://platform.openai.com/docs/guides/embeddings
    - **E5** : Microsoft의 다국어 임베딩 모델
        - https://huggingface.co/intfloat/multilingual-e5-large
    - **BGE** : [중국 BAAI](https://en.wikipedia.org/wiki/Beijing_Academy_of_Artificial_Intelligence)에서 개발한 고성능 임베딩 모델
        - https://huggingface.co/BAAI/bge-m3
- 검색 방법론
    - **Dense Retrieval** : 벡터 기반 유사도 검색
    - **Sparse Retrieval** : 키워드 기반 검색 (BM25 등)
    - **Hybrid Retrieval** : Dense와 Sparse 방법의 결합

#### RAG 파이프라인
- Data Sources → Load Data → Text Split → Indexing → Query Processing → Retrieval → Generation → Post-processing

1. **문서 로딩** → PDF 읽기
2. **문서 분할** → 작은 청크로 나누기
3. **벡터화** → 임베딩으로 변환
4. **저장** → FAISS 벡터스토어에 저장
5. **검색** → 질문과 유사한 문서 찾기
6. **생성** → LLM으로 답변 생성

### 1. Document Loader 개요 (문서 로딩)
> 문서를 Load 한 후, 이를 Document 형식으로 변환하여 검색 시스템에 연결하고, 모델이 해당 정보를 효과적으로 사용할 수 있도록 합니다.
1. **데이터 로드 및 파싱**: 다양한 파일 형식(예: PDF, CSV, 텍스트 파일 등)에서 데이터를 읽고, 이를 **Document** 객체로 변환합니다. 이 객체는 텍스트 데이터와 관련된 메타 데이터를 포함할 수 있습니다.
2. **문서 분할(Document Splitting)**: 로드 된 문서가 길면, 이를 **텍스트 블록** 단위로 나누어 검색 효율성을 높이고, 이후 모델이 사용할 수 있도록 준비합니다. 각 블록은 특정 메타데이터(예: 문서의 제목, 작성 날짜 등)를 가질 수 있습니다.
3. **검색 시스템과 연결**: Document Loader는 Load 된 문서를 **검색 인덱스**에 연결하여 RAG 시스템에서 사용할 수 있도록 합니다. 문서 로딩 후, **검색(retrieval)**이 가능하도록 필요한 데이터를 제공합니다.

##### 구성요소
1. **파일 소스**: Document Loader는 다양한 파일 형식과 데이터 소스를 처리할 수 있습니다.
    - 텍스트 파일
    - PDF 파일
    - HTML 웹 페이지
    - 데이터베이스
    - API에서 받아오는 JSON 파일
2. **파서(Parser)**: 파일에서 읽은 데이터를 **Document** 형식으로 변환하는 Parser가 필요합니다. 예를 들어, PDF에서 텍스트를 추출하려면 PDF 파서가 필요합니다.
3. **분할기(Splitter)**: 큰 문서를 효율적으로 처리하기 위해, 텍스트를 **문단 단위**, **문장 단위**, 또는 **고정 길이 블록**으로 분할 합니다. 이를 통해 검색의 효율성을 높입니다.
4. **출력 포맷**: Document Loader는 로드된 데이터를 **Document** 형식으로 반환합니다. 이 형식은 텍스트와 관련 메타 데이터를 포함하며, 이후 **Retrieval** 및 **Generation** 단계에서 활용될 수 있습니다.

종류
- **TextLoader**: 단순한 텍스트 파일을 읽어 **Document** 형식으로 변환합니다.
    
    https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.text.TextLoader.html
    
- **PDFLoader**: PDF 파일에서 텍스트를 추출하여 **Document** 형식으로 변환합니다.
    
    https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html
    
- **CSVLoader**: CSV 파일을 읽어 각 행을 **Document** 객체로 변환합니다.
    
    https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.csv_loader.CSVLoader.html
    
- **HTMLLoader**: 웹 페이지의 HTML 파일을 로드하여 텍스트 데이터를 추출합니다.
    
    https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.mhtml.MHTMLLoader.html
    
- **JSONLoader**: JSON 형식의 데이터를 로드하고 파싱합니다.
    
    https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.json_loader.JSONLoader.html

### 2. RecursiveCharacterTextSplit(문서 분할)
> 문서를 작은 청크로 분할하여 긴 문서를 LLM이 처리할 수 있는 크기로 분할

- **동작 방식:**
1. **계층적 분할**: 큰 구분자부터 시도 (`\n\n` → `\n` → `.` →  ``)
2. **의미 보존**: 문단, 문장 단위로 자연스럽게 분할
3. **중복 유지**: `chunk_overlap`으로 맥락 연결성 보존

**장점:**

- 의미 단위로 자연스러운 분할
- 맥락 손실 최소화
- 다양한 언어/형식 지원

외에도
- **CharacterTextSplitter** : 특정 기호 기준으로 나눌 때 (간단한 텍스트 분할)
    - 분할 전략
        - 의미 단위: 마침표(.) - 문장 보존
        - 구조 단위: 줄바꿈(\n) - 문단 보존
        - 단어 단위: 공백( ) - 세밀한 분할
- **RecursiveCharacterTextSplitter** : 문장 / 단락 유지하며 크기조절 (LLM에 적합)
    - 분할 전략
        - 문단 → 문장 → 단어 순으로 분할을 시도해 문맥 유지
        - 연구 논문, 기술 문서처럼 구조가 명확하지 않은 긴 텍스트를 효과적으로 나눌 때 유용함
- **TokenTextSplitter**: LLM의 토큰단위로 조절 (OpenAI, tiktoken 기반 토큰화)
    - 분할 전략
        - GPT 계열 모델처럼 토큰 단위로 입력을 받는 모델을 사용시 유리
- **Hugging Face**: AI모델의 깃허브
    - Model Hub: GPT, BERT 등 다양한 AI 모델 저장소
    - Transformers:	NLP 모델을 쉽게 사용 가능
    - Datasets:	공개 AI 데이터셋 제공
    - Tokenizers: 고성능 토크나이저 지원
    - Spaces: AI 모델을 웹 애플리케이션으로 배포

### 3. OpenAIEmbeddings (벡터화)
> 문서 분할 단계에서 생성된 문서 단위를 기계가 이해할 수 있는 수치적 형태(벡터)로 변환하는 과정

> 텍스트를 숫자 벡터로 변환, 의미가 비슷한 텍스트는 비슷한 벡터값

의미 검색(Semantic Search)

- 벡터 표현을 활용하여 의미적으로 유사한 텍스트를 검색하는 방식
- 사용자가 입력한 쿼리에 대해 가장 관련성이 높은 문서나 정보를 효과적으로 찾아낼 수 있음

문서 분류(Document Classification)

- 임베딩된 텍스트 벡터를 사용하여 문서를 특정 카테고리나 주제로 분류하는 작업
- 뉴스 분류, 고객 피드백 분석 등 다양한 자연어 처리(NLP) 응용 가능

텍스트 유사도 계산(Text Similarity Calculation)

- 두 개의 텍스트 벡터 사이의 거리를 계산하여 유사도를 평가
- 예를 들어, 코사인 유사도(Cosine Similarity)를 활용하여 두 문장이 얼마나 비슷한지 수치화할 수 있음
### 벡터 유사도

- 비슷한 의미 → 비슷한 벡터 → 높은 유사도
- 다른 의미 → 다른 벡터 → 낮은 유사도

### **장점**

- 다국어 지원 (한국어 우수)
- 의미론적 유사성 정확도 높음
- OpenAI API 간편 사용

키 없을 시 HuggingFaceEmbedding 사용

### FAISS 벡터 DB + RetrievalQA(저장 + 검색)
> 벡터 유사도 검색을 위한 고성능 라이브러리

- **내부 동작**
1. 각 문서를 임베딩으로 벡터화
2. FAISS 인덱스 생성
3. 벡터들을 인덱스에 저장
4. 메타데이터(페이지, 소스) 함께 저장

#### as_retriever() 함수
- **벡터스토어를 검색기(Retriever)로 변환**하는 메서드
- RAG 체인에서 사용할 수 있는 표준 인터페이스 제공

### RetrievalQA
> 사용자 질문 → Retriever → 관련 문서 검색 → LLM → 문서 기반 답변
- **검색(Retrieval) + 질의응답(QA)을 결합**한 LangChain의 핵심 체인
- 문서 검색과 답변 생성을 자동으로 연결해주는 **RAG의 핵심**

**1. Chain Type (처리 방식)**

- **"stuff"**: 모든 문서를 한번에 처리 (⭐ 추천)
- "map_reduce": 문서별 처리 후 통합
- "refine": 순차적 답변 개선
- "map_rerank": 최고 답변 선택

**Retriever가 RAG에서 중요한 이유:**
- 벡터스토어와 QA체인을 **연결하는 브리지** 역할을 합니다. 
- 질문에 맞는 최적의 문서들을 찾아서 LLM에게 전달하는 핵심 컴포넌트입니다.

**Vector Store Retriever vs Multi Query Retriever 비교**
- **Vector Store Retriever**는 **단순하고 빠른 검색**이 필요할 때 적합함
- **Multi Query Retriever**는 **검색 결과의 다양성을 높이고** 싶을 때 유용함